In [2]:
import pandas as pd
data=pd.read_csv('Test.csv')

##Latitude and Longitude added in the cleaned version##

In [3]:
data.head(5)

,postalcode,borough,neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.7710,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [4]:
data.dtypes

postalcode        object
borough           object
neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

##Adding the required library##

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##Got the lati and logi values of toronto##

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##Create a map of Toronto with neighborhoods superimposed on top##

In [8]:
maptoronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['borough'], data['neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        ).add_to(maptoronto)  
    
maptoronto

 ##Analysing only Downtown Toronto##

In [10]:
toronto_data = data[data['borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data

,postalcode,borough,neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.6680,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789
5,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
6,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
7,M5G,Downtown Toronto,Central Bay Street,43.6580,-79.3874
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.6506,-79.3846
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.6408,-79.3818


In [11]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="SB_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


##Clustering neighbourhoods in downtown toronto##

In [14]:
mapscar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['neighbourhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        ).add_to(mapscar)  
    
mapscar

##Pulling requests from FourSquare##

In [32]:
CLIENT_ID = 'XU1CZZNHIVN05NFBF2S45EDHMHVARE4DVNAINNSCP1FYVRN0' # your Foursquare ID
CLIENT_SECRET = '0PNSSFYZXGPPKMXD4DNUTI2Q23RUWIRCDIQSOOBSSDUO0L0E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XU1CZZNHIVN05NFBF2S45EDHMHVARE4DVNAINNSCP1FYVRN0
CLIENT_SECRET:0PNSSFYZXGPPKMXD4DNUTI2Q23RUWIRCDIQSOOBSSDUO0L0E


In [33]:
toronto_data.loc[10, 'neighbourhood']

'Design Exchange,Toronto Dominion Centre'

In [34]:
neighborhood_latitude = toronto_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[1, 'neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cabbagetown,St. James Town are 43.6472, -79.3816.


In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=XU1CZZNHIVN05NFBF2S45EDHMHVARE4DVNAINNSCP1FYVRN0&client_secret=0PNSSFYZXGPPKMXD4DNUTI2Q23RUWIRCDIQSOOBSSDUO0L0E&v=20180605&ll=43.6472,-79.3816&radius=500&limit=100'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5bbb361ed2193b451e753c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 145,
  'suggestedBounds': {'ne': {'lat': 43.6517000045, 'lng': -79.37539272984121},
   'sw': {'lat': 43.642699995499996, 'lng': -79.3878072701588}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05df964a52059f620e3',
       'name': 'Canoe',
       'location': {'address': '66 Wellington St West',
        'crossStreet': 'at Bay Street',
        'lat': 43.647452066183476,
        'lng': -79.38132001815676,
        'labeledLatLng

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canoe,Restaurant,43.647452,-79.381320
1,Equinox Bay Street,Gym,43.648100,-79.379989
2,The Fairmont Royal York,Hotel,43.645449,-79.381508
3,Maman,Café,43.648309,-79.382253
4,King Taps,Gastropub,43.648476,-79.382058


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##Creating a df to map venues in downtown toronto##

In [41]:
torontodowntown_venues = getNearbyVenues(names=toronto_data['neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [42]:
print(torontodowntown_venues.shape)
torontodowntown_venues.head()

(1282, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.6796,-79.3775,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.6796,-79.3775,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.6796,-79.3775,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.6796,-79.3775,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.6680,-79.3677,Cranberries,43.667843,-79.369407,Diner


In [43]:
print('There are {} uniques categories.'.format(len(torontodowntown_venues['Venue Category'].unique())))

There are 208 uniques categories.


##Analyze Each Neighborhood## and performing onehotencoding

In [44]:
# one hot encoding
torontodowntownhot_onehot = pd.get_dummies(torontodowntown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontodowntownhot_onehot['Neighborhood'] = torontodowntown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontodowntownhot_onehot.columns[-1]] + list(torontodowntownhot_onehot.columns[:-1])
torontodowntownhot_onehot = torontodowntownhot_onehot[fixed_columns]

torontodowntownhot_onehot.tail()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
1277,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Christie,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1278,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Christie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [45]:
torontodowntownhot_onehot.shape

(1282, 208)

In [46]:
torontodowntown_grouped = torontodowntownhot_onehot.groupby('Neighborhood').mean().reset_index()
torontodowntown_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.03,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.00,0.000000,0.01,0.020000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.010000,

In [47]:
torontodowntown_grouped.shape

(18, 208)

In [48]:
num_top_venues = 5

for hood in torontodowntown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torontodowntown_grouped[torontodowntown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.05
2    Cocktail Bar  0.05
3          Bakery  0.04
4  Farmers Market  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3          Boutique  0.07
4           Airport  0.07


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.06
2         Park  0.06
3          Pub  0.04
4  Pizza Place  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4                 Bar  0.04




##Putting into Pandas DataFrame##

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontodowntown_grouped['Neighborhood']

for ind in np.arange(torontodowntown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontodowntown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Asian Restaurant,Gym,Hotel,Clothing Store
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Café,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
3,"Cabbagetown,St. James Town",Coffee Shop,Park,Restaurant,Bakery,Pub,Italian Restaurant,Café,Pizza Place,Sandwich Place,Breakfast Spot
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop


##Clustering Neighbourhoods##

In [105]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = torontodowntown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 4, 5, 2, 0, 4, 4])

In [106]:
toronto_data.head(10)

,postalcode,borough,neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.6680,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789
5,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
6,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
7,M5G,Downtown Toronto,Central Bay Street,43.6580,-79.3874
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.6506,-79.3846
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.6408,-79.3818


In [107]:
neighborhoods_venues_sorted.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Asian Restaurant,Gym,Hotel,Clothing Store
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Café,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
2,2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
3,3,"Cabbagetown,St. James Town",Coffee Shop,Park,Restaurant,Bakery,Pub,Italian Restaurant,Café,Pizza Place,Sandwich Place,Breakfast Spot
4,3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop
5,0,"Chinatown,Grange Park,Kensington Market",Bar,Café,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Dim Sum Restaurant
6,4,Christie,Café,Grocery Store,Park,Coffee Shop,Baby Store,Diner,Italian Restaurant,Nightclub,Restaurant,Convenience Store
7,0,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Men's Store,Café,Mediterranean Restaurant
8,0,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,3,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Japanese Restaurant


##Exported files and used excel to lookup the co-ordinates##

In [303]:
from pandas import ExcelWriter

writer = ExcelWriter('toronto_data.xlsx')
toronto_data.to_excel(writer,'Sheet5')
writer.save()

In [305]:
from pandas import ExcelWriter

writer = ExcelWriter('neighborhoods_venues_sorted.xlsx')
neighborhoods_venues_sorted.to_excel(writer,'Sheet5')
writer.save()

In [108]:
import pandas as pd
Cluster=pd.read_csv('ClusterData.csv')

In [109]:
import numpy as np
Cluster = Cluster.replace(np.nan, 'Cafe', regex=True)

##Downtown Toronto Neighbourhood cluster data frame##

In [110]:
Cluster

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",43.6506,-79.3846,Coffee Shop,Cafe,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Asian Restaurant,Gym,Hotel,Clothing Store
1,Berczy Park,43.6448,-79.3733,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cafe,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.6290,-79.3944,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
3,"Cabbagetown,St. James Town",43.6680,-79.3677,Coffee Shop,Park,Restaurant,Bakery,Pub,Italian Restaurant,Cafe,Pizza Place,Sandwich Place,Breakfast Spot
4,Central Bay Street,43.6580,-79.3874,Coffee Shop,Cafe,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop
5,"Chinatown,Grange Park,Kensington Market",43.6532,-79.4001,Bar,Cafe,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Dim Sum Restaurant
6,Christie,43.6695,-79.4226,Cafe,Grocery Store,Park,Coffee Shop,Baby Store,Diner,Italian Restaurant,Nightclub,Restaurant,Convenience Store
7,Church and Wellesley,43.6659,-79.3832,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Men's Store,Cafe,Mediterranean Restaurant
8,"Commerce Court,Victoria Hotel",43.6482,-79.3798,Coffee Shop,Cafe,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,"Design Exchange,Toronto Dominion Centre",43.6472,-79.3816,Coffee Shop,Hotel,Cafe,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Japanese Restaurant


##Added cluster Lables##

In [111]:
Cluster.insert(0, 'Cluster Labels', kmeans.labels_)

In [112]:
Cluster

,Cluster Labels,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",43.6506,-79.3846,Coffee Shop,Cafe,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Asian Restaurant,Gym,Hotel,Clothing Store
1,0,Berczy Park,43.6448,-79.3733,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cafe,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
2,3,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.6290,-79.3944,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
3,0,"Cabbagetown,St. James Town",43.6680,-79.3677,Coffee Shop,Park,Restaurant,Bakery,Pub,Italian Restaurant,Cafe,Pizza Place,Sandwich Place,Breakfast Spot
4,4,Central Bay Street,43.6580,-79.3874,Coffee Shop,Cafe,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop
5,5,"Chinatown,Grange Park,Kensington Market",43.6532,-79.4001,Bar,Cafe,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Dim Sum Restaurant
6,2,Christie,43.6695,-79.4226,Cafe,Grocery Store,Park,Coffee Shop,Baby Store,Diner,Italian Restaurant,Nightclub,Restaurant,Convenience Store
7,0,Church and Wellesley,43.6659,-79.3832,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Men's Store,Cafe,Mediterranean Restaurant
8,4,"Commerce Court,Victoria Hotel",43.6482,-79.3798,Coffee Shop,Cafe,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,4,"Design Exchange,Toronto Dominion Centre",43.6472,-79.3816,Coffee Shop,Hotel,Cafe,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Japanese Restaurant


##Code to visualize the map##

In [113]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster['Latitude'], Cluster['Longitude'], Cluster['Neighborhood'], Cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#Examine Cluster 4#

In [115]:
Cluster.loc[Cluster['Cluster Labels'] == 4, Cluster.columns[[1] + list(range(5, Cluster.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Cafe,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop
8,"Commerce Court,Victoria Hotel",Cafe,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,"Design Exchange,Toronto Dominion Centre",Hotel,Cafe,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Japanese Restaurant
10,"First Canadian Place,Underground city",Cafe,Hotel,Restaurant,American Restaurant,Gastropub,Bar,Seafood Restaurant,Gym,Asian Restaurant
12,"Harbourfront East,Toronto Islands,Union Station",Aquarium,Hotel,Cafe,Pizza Place,Restaurant,Scenic Lookout,Brewery,Italian Restaurant,Sports Bar


##Examine cluster 1##

In [122]:
Cluster.loc[Cluster['Cluster Labels'] == 1, Cluster.columns[[1] + list(range(5, Cluster.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,Playground,Trail,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


#Examine cluster 0##

In [121]:
Cluster.loc[Cluster['Cluster Labels'] == 0, Cluster.columns[[1] + list(range(5, Cluster.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Cafe,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Asian Restaurant,Gym,Hotel,Clothing Store
1,Berczy Park,Cocktail Bar,Restaurant,Seafood Restaurant,Cafe,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
3,"Cabbagetown,St. James Town",Park,Restaurant,Bakery,Pub,Italian Restaurant,Cafe,Pizza Place,Sandwich Place,Breakfast Spot
7,Church and Wellesley,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Men's Store,Cafe,Mediterranean Restaurant
13,"Harbourfront,Regent Park",Bakery,Park,Pub,Mexican Restaurant,Cafe,Theater,Restaurant,Breakfast Spot,Event Space
15,"Ryerson,Garden District",Clothing Store,Cafe,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Thai Restaurant,Bar,Diner,Plaza
16,St. James Town,Restaurant,Hotel,Cafe,Clothing Store,Bakery,Gastropub,Breakfast Spot,Cosmetics Shop,Italian Restaurant
17,Stn A PO Boxes 25 The Esplanade,Restaurant,Cafe,Italian Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Pub,Hotel


In [119]:
Cluster.loc[Cluster['Cluster Labels'] == 4, Cluster.columns[[1] + list(range(5, Cluster.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Cafe,Italian Restaurant,Bar,Burger Joint,Sandwich Place,Salad Place,Chinese Restaurant,Spa,Bubble Tea Shop
8,"Commerce Court,Victoria Hotel",Cafe,Hotel,Restaurant,American Restaurant,Steakhouse,Gastropub,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,"Design Exchange,Toronto Dominion Centre",Hotel,Cafe,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Japanese Restaurant
10,"First Canadian Place,Underground city",Cafe,Hotel,Restaurant,American Restaurant,Gastropub,Bar,Seafood Restaurant,Gym,Asian Restaurant
12,"Harbourfront East,Toronto Islands,Union Station",Aquarium,Hotel,Cafe,Pizza Place,Restaurant,Scenic Lookout,Brewery,Italian Restaurant,Sports Bar


##Final Observations
1) In comparison to other clusters, cluster one has few neighbourhoods and thus less frequency of small business outlets
Note: Cluster 5  is located near the aiport and has fewest number of small business outlets
